In [1]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
# <---- You will also likely want to use the "folder" icon to add some files
#       for the agent to look at
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

NotebookAccessError: Notebook does not have access to secret OPENAI_API_KEY

In [ ]:
import json
import time
import traceback
import inspect
from litellm import completion
from dataclasses import dataclass, field
from typing import get_type_hints, List, Callable, Dict, Any

In [ ]:
tools = {}
tools_by_tag = {}